In [2]:
import json

In [3]:
from elasticsearch import Elasticsearch

In [31]:
from groq import Groq
from openai import OpenAI

In [45]:
import tiktoken

In [32]:
es_client = Elasticsearch('http://localhost:9200')

In [35]:
# client = Groq()
client = OpenAI()

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/Ax8iKcxdQxucqFHRYftdqg] already exists')

In [8]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [9]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
from tqdm.auto import tqdm

/Users/waithira/.virtualenvs/llm-zoomcamp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
for doc in documents:
    es_client.index(index=index_name, document=doc)

In [12]:
query = 'How do I execute a command in a running docker container?'

In [13]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [18]:
result_docs = elastic_search(query)

In [51]:
def build_prompt(query, context):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    # context_template = """
    # Q: {question}
    # A: {text}
    # """.strip()

    # prompt_template = """    
    # You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    # Use only the facts from the CONTEXT when answering the QUESTION.
    # If the CONTEXT doesn't contain the answer, output NONE
    
    # QUESTION: {question}
    
    # CONTEXT:
    # {context}
    # """.strip()

    context = ""

    for doc in result_docs:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
        # context_template = """
        # Q: {question}
        # A: {text}
        # """.strip()
        # context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    encoding = tiktoken.encoding_for_model("gpt-4o")

    prompt = prompt_template.format(question=query, context=context).strip()
    tokens = encoding.encode(prompt)
    print(len(tokens))
    return prompt
    

In [52]:
query = "How do I execute a command in a running docker container?"
search_results = elastic_search(query)
prompt = build_prompt(query, search_results)
llm(prompt)

326
<Encoding 'o200k_base'>


In [49]:
def llm(prompt):
    response = client.chat.completions.create(
        model = "gpt-4o",
        # model = 'meta-llama/llama-4-scout-17b-16e-instruct',
        messages = [{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [37]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [38]:
rag(query)

1478
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: How do I execute a command in a running docker container?

    CONTEXT:
    Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_di

NameError: name 'tiktoken' is not defined